<a href="https://colab.research.google.com/github/FlowAlpha/SPIRES_kenya_sensing/blob/main/benin_export_tiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook. The model is a [fully convolutional neural network (FCNN)](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf), specifically [U-net](https://arxiv.org/abs/1505.04597). This notebook shows:

1.   Exporting training/testing patches from Earth Engine, suitable for training an FCNN model.
2.   Preprocessing.
3.   Training and validating an FCNN model.
4.   Making predictions with the trained model and importing them to Earth Engine.

This notebook implements a UNet model for prediction in Benin from Landsat7 data. Major code chunks borrowed from.


1.   https://csaybar.github.io/blog/2019/06/21/eetf2/
2.   https://developers.google.com/earth-engine/guides/tf_examples

# Setup software libraries

Authenticate and import as necessary.

IMPORTANT: When training make sure that you are connected to a GPU runtime. When uploading data you do not have to be. 

In [ ]:
#After running this cell, restart your kernel and run it again to properly import geemap
#import subprocess

#try:
#    import geemap
#except ImportError:
#    print('Installing geemap ...')
#    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
#    !pip install earthengine-api

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)
# Folium setup.
import folium
print(folium.__version__)
import numpy as np

## Specify your Cloud Storage Bucket
You must have write access to a bucket to run this demo!  To run it read-only, use the demo bucket below, but note that writes to this bucket will not work.

In [ ]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'jp_bucket_1'

## Set other global variables

Specify the bands we want to use for prediction. Here we select R,G,B values as well as bands to calculate the NDVI index with. Select kernel size based on the tile size you would like your model to train on

In [ ]:
opticalBands = ['B3','B2','B1'] #RGB
thermalBands = ['B4','B3'] #NIR

# Specify inputs (Landsat bands) to the model and the response variable.

BANDS = ['R', 'G', 'B', 'NDVI']
RESPONSE = 'target'
FEATURES = BANDS + [RESPONSE] 

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 128
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))



#Download Benin Data and create Raster of villages

#Import Voronoi Raster here:
voronoi = ee.FeatureCollection('projects/sanford-project-04a9/assets/voronoi_villages')

treated_voronoi = voronoi.filter(ee.Filter.eq('treated', 1))

#Get Benin Feature collection
benin = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na','Benin')).set('ORIG_FID',0)
treated_voronoi_rast = treated_voronoi.filter(ee.Filter.notNull(['VID'])).reduceToImage(properties=['VID'],reducer= ee.Reducer.first())

# Create a village mask based on the treated village raster
villagemask = treated_voronoi_rast.mask() #select(['ORIG_FID']).gt(.1)


# Imagery

Gather and setup the imagery to use for inputs (predictors).  This is a three-year, cloud-free, Landsat 7 composite. Landsat 7 contains imagery pre 2009 lottery.  Display it in the notebook for a sanity check.

In [ ]:
#Get sample points we will train on
#sample_points = ee.FeatureCollection('projects/sanford-project-04a9/assets/128_pixel_tiles').sort('random')
#Get centroids of patches we want to pull from
sample_points = ee.FeatureCollection("projects/sanford-project-04a9/assets/128_pixel_centroids")
#64_pixel_tiles_only_villages is only the tiles that are 100% within a village so they don’t require any masking and there are only 4494 of them

In [ ]:
#Display Grids that we will use for training
sample_points
mapid = sample_points.getMapId({'bands': ['R', 'G', 'B'], 'min': 0, 'max': 3000})
map = folium.Map(location=[9.8, 2.4], zoom_start = 7)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
##Import Benin Imagery to make predictions
def cloudMaskL457(image): 
  qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

#Select Geometry from all of Benin or just treatment and control circles

#This geometry selects from all of Benin
image = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterDate('2007-01-01', '2008-12-31')
#image = (image.map(cloudMaskL457).median().clip(benin.geometry().buffer(10000)).reproject(crs = ee.Projection('EPSG:32631'), scale=30))
#image = (image.map(cloudMaskL457).median().setDefaultProjection(image.first().projection()).clip(benin.geometry().buffer(10000)))
image = (image.map(cloudMaskL457).median().clip(benin.geometry().buffer(10000)))

#This geometry selects only from villages
#image = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterDate('2005-01-01', '2006-12-31').map(cloudMaskL457).median().clip(voronoi.geometry())

#This geometry selects only from list of evenly spaced points created by Luke
#image = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterDate('2005-01-01', '2006-12-31').map(cloudMaskL457).filterBounds(sample_points).median()


image_ndvi = image.normalizedDifference(thermalBands).rename(['NDVI'])
image_rgb = image.select(opticalBands).rename(['R','G','B']) 
image = image_rgb.addBands(image_ndvi)
image = image.unmask(0)


In [ ]:

#Running this code will display the imagery
mapid = image.getMapId({'bands': ['R', 'G', 'B'], 'min': 0, 'max': 3000})
map = folium.Map(location=[9.8, 2.4], zoom_start = 7)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Below is using a version of cloud masking from Landsat8. We should stick to the version adapted to Landsat7 previously, although in some cases the landsat8 cloud masking looks better.

In [ ]:
# Use Landsat 7 surface reflectance data.
# l7sr = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")

# # Cloud masking function. From Landsat 8 but may want to find a filtering function specifically for Landsat 7
# def maskL8sr(image):
#   cloudShadowBitMask = ee.Number(2).pow(3).int()
#   cloudsBitMask = ee.Number(2).pow(5).int()
#   qa = image.select('pixel_qa')
#   mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
#     qa.bitwiseAnd(cloudsBitMask).eq(0))
#   mask2 = image.mask().reduce('min')
#   mask3 = image.select(opticalBands).gt(0).And(
#           image.select(opticalBands).lt(10000)).reduce('min')
#   mask = mask1.And(mask2).And(mask3)
#   return image.select(opticalBands).divide(10000).addBands(
#           image.select(thermalBands).divide(10).clamp(273.15, 373.15)
#             .subtract(273.15).divide(100)).updateMask(mask)

# # The image input data is a cloud-masked median composite. Get data from pre 2009.
# image_2 = l7sr.filterDate('2005-01-01', '2006-12-31').map(maskL8sr).median().clip(benin.geometry().buffer(3000))

# image_2_ndvi = image_2.normalizedDifference(thermalBands).rename(['NDVI'])
# image_2_rgb = image_2.select(opticalBands).rename(['R','G','B']) 
# image_2 = image_2_rgb.addBands(image_ndvi)





# #Use folium to visualize the imagery.
# mapid = image_2.getMapId({'bands': ['R', 'G', 'B'], 'min': 0, 'max': .3})
# map = folium.Map(location=[9.8, 2.4], zoom_start = 7)
# folium.TileLayer(
#     tiles=mapid['tile_fetcher'].url_format,
#     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     overlay=True,
#     name='median composite',
#   ).add_to(map)
# map.add_child(folium.LayerControl())
# map
#Optional to add NDVI Visualization
# mapid = image.getMapId({'bands': ['NDVI'], 'min': 0, 'max': 0.5})
# folium.TileLayer(
#     tiles=mapid['tile_fetcher'].url_format,
#     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     overlay=True,
#     name='thermal',
#   ).add_to(map)
# map.add_child(folium.LayerControl())
# map

Prepare the response (what we want to predict).  This is either treatment if we are looking to estimate propensity score or landcover in 2020. For predicting treatment we add on a target band marked as 1 within villages. For landcover in 2020 we add in bands specifying the land type. Only run one code chunk and comment out the other.

In [ ]:
#Run this Code if you'd like to export the training imagery to google earth enginge for visualization.
# import time 
# # Export the image to an Earth Engine asset.
# task = ee.batch.Export.image.toAsset(**{
#   'image': image,
#   'description': 'imageToAssetExample',
#   'assetId': 'users/jacksonpullman/BeninImagery',
#   'scale': 100,
#   'region': benin.geometry().getInfo()['coordinates']
# })
# task.start()
# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(5)

In [ ]:
#Code to add in bands for propensity score prediction 
l7Masked = image.updateMask(villagemask)
l7Unmasked = l7Masked.unmask(-9999)
outside_circle = 'b("R") > -9000'
target = l7Unmasked.expression(outside_circle).rename("target")


#Code to add in bands for landcover prediction
#target = ee.ImageCollection("ESA/WorldCover/v100").max().rename('target').divide(100).float().clip(benin.geometry().buffer(3000))



In [ ]:
#Visualize imagery
mapid = target.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[9.8, 2.4], zoom_start = 7)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='target variable',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Stack the 2D images (Landsat composite and response) to create a single image from which samples can be taken.  Convert the image into an array image in which each pixel stores 64x64 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [ ]:
#Combine response and image band values for training
featureStack = ee.Image.cat([
  image.select(BANDS),
  target.select(RESPONSE),
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

Use some pre-made geometries to sample the stack in strategic locations.  Specifically, these are hand-made polygons in which to take the 256x256 samples.  Display the sampling polygons on a map, red for training polygons, blue for evaluation.

# Sampling

The mapped data look reasonable so take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data for the FCNN, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export. THIS CODE CAN TAKE A WHILE TO RUN (10 minutes) MAKE SURE YOU CHECK THE FILES ARE UPLOADED TO GOOGLE CLOUD BEFOE CONTINUING.

In [ ]:
# Specify names locations for outputs in Cloud Storage. 
FOLDER = 'unet_2-23_128_tiles_benin_villages_r_g_b_ndvi_treatment'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

In [ ]:
import numpy as np
import time

def saveCNN_batch(image, point,kernel_size,scale,FilePrefix, selectors,folder, bucket= BUCKET):
  """
    Export a dataset for semantic segmentation by batches
  
  Params:
  ------
    - image : ee.Image to get pixels from; must be scalar-valued.
    - point : Points to sample over.
    - kernel_size : The kernel specifying the shape of the neighborhood. Only fixed, square and rectangle kernels are supported.
      Weights are ignored; only the shape of the kernel is used.
    - scale : A nominal scale in meters of the projection to work in.
    - FilePrefix : Cloud Storage object name prefix for the export.
    - selector : Specified the properties to save.
    - bucket : The name of a Cloud Storage bucket for the export.  
  """
  print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + bucket) 
    else 'Output Cloud Storage bucket does not exist.')
  
  # Download the points (Server -> Client)
  nbands = len(selectors)
  points = point.geometry().getInfo()['coordinates']    
  nfeatures = kernel_size*kernel_size*nbands*len(points) #estimate the totals # of features
     
  image_neighborhood = arrays
  filenames = []
  
  #Threshold considering the max number of features permitted to export.
  if nfeatures > 3e6:
    nparts = int(np.ceil(nfeatures/3e6))
    print('Dataset too long, splitting it into '+ str(nparts),'equal parts.')
    
    nppoints = np.array(points)
    np.random.shuffle(nppoints)
    
    count_batch = 1  # Batch counter 
    
    for batch_arr in np.array_split(nppoints,nparts):
      
      fcp = ee.FeatureCollection([
          ee.Feature(ee.Geometry.Point(p),{'class':'NA'}) 
          for p in batch_arr.tolist() 
      ])
      
      # Agriculture dataset (fcp-points) collocation to each L5 grid cell value.
      train_db = image_neighborhood.sampleRegions(collection=fcp, scale=scale)

      if(count_batch == 1):
        full_train_db = train_db
      else:
        full_train_db.merge(train_db)



      filename = '%s/%s-%04d_' % (folder,FilePrefix,count_batch)
      
      # Create the tasks for passing of GEE to Google storage
    print('sending the task #%04d'%count_batch)
    Task = ee.batch.Export.table.toCloudStorage(
        collection=full_train_db,        
        selectors=selectors,          
        description='Export batch '+str(count_batch),
        fileNamePrefix=filename,
        bucket=bucket,  
        fileFormat='TFRecord')
      
    Task.start()
    filenames.append(filename)
    count_batch+=1
      
    while Task.active():
      print('Polling for task (id: {}).'.format(Task.id))
      time.sleep(3)
        
    return filenames
  
#  else:    
#    train_db = image_neighborhood.sampleRegions(collection=points, scale=scale)         
#    Task = ee.batch.Export.table.toCloudStorage(
#      collection=train_db,
#      selectors=selectors,
#      description='Training Export',
#      fileNamePrefix=FilePrefix,
#      bucket=bucket,  
#      fileFormat='TFRecord')
#    Task.start()
#    
#    while Task.active():
#      print('Polling for task (id: {}).'.format(Task.id))
#      time.sleep(3)
    
#    return FilePrefix

Want to update this to upload feature collection in bulk, instead of 60,000 separate items.

In [ ]:
#Divide training and testing points
train_points = sample_points.filter(ee.Filter.lt('random', 0.66))
test_points = sample_points.filter(ee.Filter.gt('random', 0.66))
print(train_points.size().getInfo())
print(test_points.size().getInfo())

In [ ]:
selectors = ['R','G','B','NDVI','target']
train_filenames = saveCNN_batch(featureStack,train_points,128,30,'trainUNET', selectors,folder =FOLDER)
test_filenames = saveCNN_batch(featureStack,test_points,128,30,'testUNET', selectors,folder =FOLDER)
#Occasionannly computed value is too large, so we will chunk our upload into groups of 300 tiles

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.  The following are helper functions for that.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)




def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]



def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [ ]:
BUFFER_SIZE = 10
BATCH_SIZE = 16

In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns: 
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + "train" + '*'
	#glob = 'gs://' + BUCKET + '/' + "unet_10-28" + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	#dataset = dataset.batch(BATCH_SIZE)
	#dataset = dataset.batch(1).repeat()
	return dataset

training = get_training_dataset()

print(iter(training.take(1)).next())

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns: 
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + "test" + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

In [ ]:
print(iter(evaluation.take(1)).next())

# Model

Here we use the Keras implementation of the U-Net model.  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class probability, label or a continuous output. Depending on what we are trying to predict (treatment or land cover) we can choose to treat this as a classification or regression problem. For propensity score estimation here we choose to use a custom Binary Cross Entropy Dice Loss Function and for regression we can use typical Mean Square Error. Can Choose to change this.

In [ ]:
#Create Custom Losses
from tensorflow.keras import losses
from tensorflow.keras import metrics

def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
  loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
  return loss


In [ ]:
# Specify model training parameters.
#OPTIMIZER = 'SGD'
OPTIMIZER = 'adam'
#LOSS = 'MeanSquaredError'
LOSS = 'BinaryCrossentropy'
#METRICS = ['RootMeanSquaredError']
METRICS = ['AUC']

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	#Change final layer activation based on what you are predicting
	#outputs = layers.Conv2D(1, (1, 1), activation='linear')(decoder0)
	outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER), 
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

# Training the model

You train a Keras model by calling `.fit()` on it.  Here we're going to train for 10 epochs, which is suitable for demonstration purposes.  For production use, you probably want to optimize this parameter, for example through [hyperparamter tuning](https://cloud.google.com/ml-engine/docs/tensorflow/using-hyperparameter-tuning).

In [ ]:
#import capabilities to save model to drive

from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import datetime

# Callbacks time
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
es = EarlyStopping(monitor='val_loss', patience=5)
mcp = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

# Run this cell to mount your Google Drive.
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
TRAIN_SIZE = train_points.size().getInfo()
EVAL_SIZE = test_points.size().getInfo()
print(EVAL_SIZE)
print(TRAIN_SIZE)

In [ ]:
m = get_model()

history = m.fit(
    x=training, 
    epochs=5, 
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE), 
    validation_data=evaluation,
    validation_steps=EVAL_SIZE)

Note that the notebook VM is sometimes not heavy-duty enough to get through a whole training job, especially if you have a large buffer size or a large number of epochs.  You can still use this notebook for training, but may need to set up an alternative VM ([learn more](https://research.google.com/colaboratory/local-runtimes.html)) for production use.  Alternatively, you can package your code for running large training jobs on Google's AI Platform [as described here](https://cloud.google.com/ml-engine/docs/tensorflow/trainer-considerations).  The following code loads a pre-trained model, which you can use for predictions right away.

In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

#!mkdir -p drive/MyDrive/unet
m.save('drive/MyDrive/unet/unet_10-30')

In [ ]:
# Load a trained model. 50 epochs. 25 hours. Final RMSE ~0.08.
#MODEL_DIR = 'gs://ee-docs-demos/fcnn-demo/trainer/model'
#m = tf.keras.models.load_model(MODEL_DIR)
#m.summary()

# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storage bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.

The following functions handle this process.  It's useful to separate the export from the predictions so that you can experiment with different models without running the export every time.

In [ ]:
image.clip(alibori)

In [ ]:
def doExport(out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(BANDS),
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'target': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.

In [ ]:
alibori = ee.FeatureCollection("projects/sanford-project-04a9/assets/Alibori_study_area")
#alibori = image.clip(alibori)

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/jacksonpullman' # INSERT YOUR FOLDER HERE.

# Base file name to use for TFRecord files and assets.
bj_image_base = '3-2-pred'
# Half this will extend on the sides of each patch.
bj_kernel_buffer = [32, 32]

bj_region = alibori.geometry()


In [ ]:
# Run the export.
doExport(bj_image_base, bj_kernel_buffer, bj_region)

In [ ]:
# Run the prediction.
doPrediction(bj_image_base, user_folder, bj_kernel_buffer, bj_region)

# Display the output

One the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the predictions over Benin. NEED TO WAIT ~30 MINUTES FOR UPLOAD TO REACH EARTH ENGINE.

In [ ]:
out_image = ee.Image(user_folder + '/' + bj_image_base).clip(alibori.geometry())
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[9.8, 2.4], zoom_start = 7)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted target',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
#Visualize Training AND Testing Villages